# Log-Normal or Over-Dispersed Poisson?

We replicate the empirical applications in [Harnau (2018a)](http://www.mdpi.com/2227-9091/6/3/70) in Section 2 and Section 6.

*The work on this vignette was supported by the European Research Council, grant AdG 694262.*

First, we import the package

In [1]:
import apc

In [2]:
# Turn off FutureWarnings
import warnings
warnings.simplefilter('ignore', FutureWarning)

## 2. Empirical illustration of the problem

This section motivates the problem. Based on the Verrall et al. (2010) it applies the misspecification tests from [Harnau (2018b)](http://www.mdpi.com/2227-9091/6/2/25). We split the data into two sub-samples after the fifth accident year. Then we test for breaks in dispersion parameters with a Bartlett test and linear predictors with an F-test.

*Remark: we replicated the empirical applications in Harnau (2018b) [here](https://github.com/JonasHarnau/apc/blob/master/apc/vignettes/vignette_misspecification.ipynb).*

In [3]:
for family in ('log_normal_response', 'od_poisson_response'):
    model_VNJ = apc.Model()
    model_VNJ.data_from_df(apc.loss_VNJ(), data_format='CL')
    model_VNJ.fit(family, 'AC')
    
    sub_models_VNJ = [model_VNJ.sub_model(coh_from_to=(1,5)),
                     model_VNJ.sub_model(coh_from_to=(6,10))]

    bartlett_VNJ = apc.bartlett_test(sub_models_VNJ)
    f_VNJ = apc.f_test(model_VNJ, sub_models_VNJ)
    
    print(family)
    print('='*len(family))
    print('Bartlett test p-value: {:.2f}'.format(
        bartlett_VNJ['p_value']))
    print('F-test p-value: {:.2f} \n'.format(
        f_VNJ['p_value']))

log_normal_response
Bartlett test p-value: 0.09
F-test p-value: 0.91 

od_poisson_response
Bartlett test p-value: 0.78
F-test p-value: 0.64 



Neither in a log-normal nor in an over-dispersed Poisson model can we convincingly reject the model specification based on these tests. This illustrates a situation in which it is not clear what model to use so that the new R-test can prove its usefulness.

## 6.1 Empirical illustration revisited

In this section, we return to the empirical illustration from above and test whether an R-test can help us to decide between (generalized) log-normal and over-dispersed Poisson model. 

The package comes with built-in functionality for the R-test. Say we want to test 
$$ H_0: \text{generalized log-normal} \quad \text{vs} \quad \text{over-dispersed Poisson} $$
based on the statistic $R^*_{ls}$ and compare it to $\widehat{\mathrm{R}}^*_{ls}$.

In [4]:
r_VNJ = apc.r_test(apc.loss_VNJ(), # specify the data set
                   family_null='gen_log_normal_response', # declare null model
                   predictor='AC', # AC = age-cohort matching the chain-ladder
                   R_stat='wls_ls', # R-stat: wls_ls -> R^{star}_{ls}
                   R_dist='wls_ls') # Pi est in limiting dist: wls_ls -> Pi^{star}_{ls}
print('R-statistic: {:.2f}'.format(r_VNJ['R_stat']))
print('p_value: {:.4f}'.format(r_VNJ['p_value']))

Inferred 'data_format' from response: CA
R-statistic: 113.19
p_value: 0.0011


This matches the value for the statistic in the paper and the p-value in Table 3 (which is given in %).

*Remark: besides the value of the test statistic and the p-value under the null, apc.r_test also return the power at the value oder the R-statistic (*```power_at_R```*). The power corresponds to one minus the p-value under the alternative.*

To replicate the remaining test statistics and the entire Table 3 we employ a small function that iterates over all possible combinations. 

In [5]:
import pandas as pd

def r_test_all_combs(model):
    # create an empty series to be filled with R-statistics
    R_stats = pd.Series(None, index=('$R_{ls}$', '$R_{ql}$', 
                                     '$R^*_{ls}$', '$R^*_{ql}$'))
    # create and empty df to be filled with p-values
    base_df = pd.DataFrame(
        None, 
        index = ('$\widehat{\mathrm{R}}_{ls}$', 
                 '$\widehat{\mathrm{R}}_{ql}$', 
                 '$\widehat{\mathrm{R}}^*_{ls}$', 
                 '$\widehat{\mathrm{R}}^*_{ql}$'),
        columns = pd.MultiIndex.from_product(
            [
                ('$H_0: $ generalized log-normal', 
                 '$H_0: $ over-dispersed Poisson'),
                ('$R_{ls}$', '$R_{ql}$', '$R^*_{ls}$', '$R^*_{ql}$')
            ])
    )
    # iterate over ways to compute the R-statistic
    for i, R_stat in enumerate(['ls', 'ql', 'wls_ls', 'wls_ql']):
        # iterate over ways to estimate Pi in the limiting dist
        for j, R_dist in enumerate(['ls', 'ql', 'wls_ls', 'wls_ql']):
            # compute R-test
            r_test = apc.r_test(apc.loss_VNJ(), 
                                family_null='gen_log_normal_response',
                                predictor='AC', 
                                R_stat=R_stat, R_dist=R_dist, 
                                data_format='CL')
            base_df.iloc[j, i] = r_test['p_value']
            base_df.iloc[j, i+4] = 1 - r_test['power_at_R']
        R_stats.iloc[i] = r_test['R_stat']
    return base_df, R_stats

table3, R_stats = r_test_all_combs(model_VNJ)

The R-statistics are as follows.

In [6]:
pd.DataFrame(R_stats.rename('R-Statistic')).T

,$R_{ls}$,$R_{ql}$,$R^*_{ls}$,$R^*_{ql}$
R-Statistic,104.869134,105.610958,113.185124,108.39224


And Table 3 is given by this:

In [7]:
table3*100

$H_0: $ generalized log-normal            \
                                                    $R_{ls}$  $R_{ql}$   
$\widehat{\mathrm{R}}_{ls}$                         0.425208  0.385867   
$\widehat{\mathrm{R}}_{ql}$                         0.316337  0.285603   
$\widehat{\mathrm{R}}^*_{ls}$                       0.351596  0.318049   
$\widehat{\mathrm{R}}^*_{ql}$                        0.38044   0.34464   

                                                     \
                              $R^*_{ls}$ $R^*_{ql}$   
$\widehat{\mathrm{R}}_{ls}$     0.142406   0.267857   
$\widehat{\mathrm{R}}_{ql}$     0.100103   0.194504   
$\widehat{\mathrm{R}}^*_{ls}$   0.113691   0.218175   
$\widehat{\mathrm{R}}^*_{ql}$   0.125013   0.237697   

                              $H_0: $ over-dispersed Poisson           \
                                                    $R_{ls}$ $R_{ql}$   
$\widehat{\mathrm{R}}_{ls}$                          8.53354  9.00184   
$\widehat{\mathrm{R}}_{ql}$                          11.8047  12.4049   
$\widehat{\mathrm{R}}^*_{ls}$                        10.4221  10.9669   
$\widehat{\mathrm{R}}^*_{ql}$                        9.48055  9.98668   

                                                     
                              $R^*_{ls}$ $R^*_{ql}$  
$\widehat{\mathrm{R}}_{ls}$      14.5894    10.8861  
$\widehat{\mathrm{R}}_{ql}$      19.3502    14.7893  
$\widehat{\mathrm{R}}^*_{ls}$    17.3429    13.1413  
$\widehat{\mathrm{R}}^*_{ql}$    15.9619    12.0142

In the paper we now move on to find the 5% critical value under the over-dispersed Poisson model as well as the power at that value. This functionality is not directly implemented in the package; however we can easily replicate it with the package [quad_form_ratio](https://github.com/JonasHarnau/quad_form_ratio). 

In [8]:
from quad_form_ratio import saddlepoint_cdf_R, saddlepoint_inv_cdf_R
import numpy as np

model_VNJ.fit('log_normal_response', 'AC')
X, Z = model_VNJ.design, np.log(model_VNJ.data_vector['response'])
tau_ls = model_VNJ.fitted_values.sum() 
sqrt_Pi_ls = np.diag(np.sqrt(model_VNJ.fitted_values/tau_ls))
RSS = model_VNJ.RSS

X_star_ls, Z_star_ls = sqrt_Pi_ls.dot(X), sqrt_Pi_ls.dot(Z) 

# fit the weighted least squares model, we set rcond=0. since
# we know that X_star has full column rank.
wls_ls_fit = np.linalg.lstsq(X_star_ls, Z_star_ls, rcond=0.)
xi_star_ls, RSS_star_ls = wls_ls_fit[0], wls_ls_fit[1][0]

fitted_wls_ls = np.exp(X.dot(xi_star_ls))
sqrt_Pi_star_ls = np.diag(np.sqrt(fitted_wls_ls/fitted_wls_ls.sum()))

# Use the QR-decomposition to compute the orthogonal projection M
Q, _ = np.linalg.qr(X)
M = np.identity(model_VNJ.n) - Q.dot(Q.T)

# do the same for the weighted least squares orthogonal projection
X_star_ls = sqrt_Pi_star_ls.dot(X)        
Q_star_ls, _ = np.linalg.qr(X_star_ls)
M_star_ls = np.identity(model_VNJ.n) - Q_star_ls.dot(Q_star_ls.T)

# A refers to the sandwiched matrix in the numerator
# B refers to the sandwiched matrix in the denominator
# _gln and _odp refer to the sandwiches under the respective nulls
A_gln = M
B_gln = sqrt_Pi_star_ls.dot(M_star_ls).dot(sqrt_Pi_star_ls)
A_odp = np.linalg.inv(sqrt_Pi_star_ls).dot(M).dot(np.linalg.inv(sqrt_Pi_star_ls))
B_odp = M_star_ls

# We compute the 5% critical value under ODP (lower quantile)
# The function iterates to find the critical value up to a precision of 0.0001
cv = saddlepoint_inv_cdf_R(A_odp, B_odp, probabilities=[0.05])
print('5% critical value for over-dispersed Poisson: {:.1f}'.format(cv[0.05]))

# Given the critical value, we compute the power
pwr_at_cv5 = saddlepoint_cdf_R(A_gln, B_gln, cv)
print('Power at 5% critical value: {:.2f}'.format(pwr_at_cv5.iloc[0]))



5% critical value for over-dispersed Poisson: 95.7
Power at 5% critical value: 0.99


## 6.2 Sensitivity to invalid model reductions

In this section, we use the data from Barnett and Zehnwirth (2000, Table 3.5). These data are known to require a calendar effect for modeling. We show that the test results may be misleading when the baseline model is already misspecified.

### $H_0:$ Generalized log-normal

First, we test in a model with calendar effect so the linear predictor is $\mu_{ij} = \alpha_i + \beta_j + \gamma_k + \delta$. The first hypothesis we consider is
$$ H_0: \text{extended generalized log-normal} \quad \text{vs} \quad H_A: \text{extended over-dispersed Poisson}.$$
This is easily tested with an $R$-test.

In [9]:
r_BZ_GLNe = apc.r_test(
    apc.loss_BZ(), family_null='gen_log_normal_response', 
    predictor='APC', # APC = age-period-cohort, incl. calendar effect
    data_format='CL' # optional, the package can infer the data_format
) # the default for R_stat and R_dist are our preferrred 'wls_ls'
print('R-statistic: {:.2f}'.format(r_BZ_GLNe['R_stat']))
print('p_value: {:.2f}'.format(r_BZ_GLNe['p_value']))

R-statistic: 114.40
p_value: 0.02


Thus, we reject the extended generalized log-normal model.

Despite the rejection of the model, we move on to test whether we can drop the calendar effect:
$$ H_0: \text{generalized log-normal} \quad \text{vs} \quad H_A: \text{extended generalized log-normal}.$$
We can do this with a simple $F$-test, both in a log-normal and a generalized log-normal model (Kuang and Nielsen, 2018).

In [10]:
model_BZ = apc.Model()
model_BZ.data_from_df(apc.loss_BZ(), data_format='CL')
model_BZ.fit_table('log_normal_response', attach_to_self=False).loc[
    ['AC'],:
]

,-2logL,df_resid,LR_vs_APC,df_vs_APC,F_vs_APC,P>F,aic
AC,-166.967,45,120.492,9,20.8271,9.59281e-12,-124.967


As expected for the data at hand, we reject this reduction; the calendar effect is needed.

For illustrative purposes, we nonetheless move on to test 
$$ H_0: \text{generalized log-normal} \quad \text{vs} \quad H_A: \text{over-dispersed Poisson}, $$
thus a scenario in which neither model has a calendar effect.

In [11]:
r_BZ_GLN = apc.r_test(
    apc.loss_BZ(), family_null='gen_log_normal_response', 
    predictor='AC', data_format='CL'
)
print('R-statistic: {:.2f}'.format(r_BZ_GLN['R_stat']))
print('p_value: {:.2f}'.format(r_BZ_GLN['p_value']))

R-statistic: 87.54
p_value: 0.10


Perhaps surprisingly, the generalized log-normal model looks better now - we cannot convincingly reject it against the over-dispersed Poisson model.

### $H_0:$ Over-dispersed Poisson

Now we start the other way around and take the over-dispersed Poisson model as a baseline. First, we again include a calendar effect and test
$$ H_0: \text{extended over-dispersed Poisson} \quad \text{vs} \quad H_A: \text{extended generalized log-normal}.$$

In [12]:
r_BZ_ODPe = apc.r_test(
    apc.loss_BZ(), family_null='od_poisson_response', data_format='CL'
) # the default for predictor is APC, thus includes a calendar effect
print('R-statistic: {:.2f}'.format(r_BZ_ODPe['R_stat']))
print('p_value: {:.2f}'.format(r_BZ_ODPe['p_value']))
print('Power at R: {:.2f}'.format(r_BZ_ODPe['power_at_R']))

R-statistic: 114.40
p_value: 0.14
Power at R: 0.98


In this case, we cannot reject the over-dispersed Poisson model. The power at the value of $R^*_{ls}$ is $0.98$; this corresponds to one minus the p-value under the extended generalized log-normal null hypothesis.

Just as before, we can test whether we can reasonably drop the calendar effect, just now from the extended over-dispersed Poisson model:
$$ H_0: \text{over-dispersed Poisson} \quad \text{vs} \quad H_A: \text{extended over-dispersed Poisson}.$$
This is easily done with an $F$-test (Harnau and Nielsen 2017).

In [13]:
model_BZ = apc.Model()
model_BZ.data_from_df(apc.loss_BZ(), data_format='CL')
model_BZ.fit_table('od_poisson_response', attach_to_self=False).loc[
    ['AC'],:
]

,deviance,df_resid,P>chi_sq,LR_vs_APC,df_vs_APC,F_vs_APC,P>F
AC,36593.9,45,NaN,32146.4,9,28.9124,6.9833e-14


One again, this reduction is rejected.

Neglecting this result, we investigate what happens if we test the models without calendar effect in
$$ H_0: \text{over-dispersed Poisson} \quad \text{vs} \quad H_A: \text{generalized log-normal}.$$

In [14]:
r_BZ_ODP = apc.r_test(
    apc.loss_BZ(), family_null='od_poisson_response', 
    predictor='AC', data_format='CL'
)
print('R-statistic: {:.2f}'.format(r_BZ_ODP['R_stat']))
print('p_value: {:.2f}'.format(r_BZ_ODP['p_value']))

R-statistic: 87.54
p_value: 0.01


This time, we reject the over-dispersed Poisson model. Thus, the results completely flipped by dropping the calendar effect. With calendar effect, we cannot reject the over-dispersed Poisson model but can reject the generalized log-normal model. By dropping the much needed calendar effect, we turn this on its head and reject the over-dispersed Poisson but not the generalized log-normal model. Thus, we should be careful what we use as a baseline model before testing.

## 6.3 A general to specific testing procedure

Taking into account the insights from above, we now consider a general to specific testing procedure. That is, we start with "the most general" model and test for possible reductions, stopping once we run into a rejection. For this application, we consider the data by Taylor and Ashe (1983) that has been become a kinda of benchmark data set.

First, we consider an extended generalized log-normal model and test it against its over-dispersed Poisson counterpart:
$$ H_0: \text{extended generalized log-normal} \quad \text{vs} \quad H_A: \text{extended over-dispersed Poisson}.$$

In [15]:
r_TA_GLNe = apc.r_test(
    apc.loss_TA(), family_null='gen_log_normal_response', 
    predictor='APC', data_format='CL'
)
print('R-statistic: {:.2f}'.format(r_TA_GLNe['R_stat']))
print('p_value: {:.4f}'.format(r_TA_GLNe['p_value']))

R-statistic: 81.54
p_value: 0.0012


The $R$-test rejects the extended generalized log-normal model. Thus, we do not proceed with this model.

Instead, we now consider the reverse test:
$$ H_0: \text{extended over-dispersed Poisson} \quad \text{vs} \quad H_A: \text{extended generalized log-normal}. $$

In [16]:
r_TA_ODPe = apc.r_test(
    apc.loss_TA(), family_null='od_poisson_response', 
    predictor='APC', data_format='CL'
)
print('R-statistic: {:.2f}'.format(r_TA_ODPe['R_stat']))
print('p_value: {:.4f}'.format(r_TA_ODPe['p_value']))
print('Power at R: {:.2f}'.format(r_TA_ODPe['power_at_R']))

R-statistic: 81.54
p_value: 0.9238
Power at R: 1.00


We cannot reject the hypothesis. Thus, we got to hunt for further evidence against the extended over-dispersed Poisson model.

We consider the misspecification tests from Harnau (2018b). In the extended over-dipsersed Poisson model with calendar effect, we split the data into four sub-samples and then test
$$ H_{\sigma^2}: \sigma^2_\ell = \sigma^2 $$
with a Bartlett test.

In [17]:
model_TAe = apc.Model()
model_TAe.data_from_df(apc.loss_TA(), data_format='CL')
model_TAe.fit('od_poisson_response', 'APC')

sub_models_TAe = [model_TAe.sub_model(per_from_to=(1,5)),
                  model_TAe.sub_model(
                      coh_from_to=(1,5), age_from_to=(1,5), per_from_to=(6,10)
                  ),
                  model_TAe.sub_model(age_from_to=(6,10)),
                  model_TAe.sub_model(coh_from_to=(6,10))]

bartlett_TA_ODPe = apc.bartlett_test(sub_models_TAe)
print('Bartlett test p-value: {:.2f}'.format(bartlett_TA_ODPe['p_value']))

Bartlett test p-value: 0.05


This is a somewhat close call. In light of the fact that simpler models tend to perform better in forecasting, we interpret the test result as the absence of strong evidence against the hypothesis.

Then, taking $H_{\sigma^2}$ as given, we can move on to test for breaks in linear predictors across sub-samples 
$$ H_{\mu, \sigma^2}: \alpha_{i, \ell} + \beta_{j, \ell} + \gamma_{k, \ell} + \delta_\ell = \alpha_i + \beta_j + \gamma_k + \delta. $$
We do so using an $F$-test.

In [18]:
f_TA_ODPe = apc.f_test(model_TAe, sub_models_TAe)
print('F-test p-value: {:.2f} \n'.format(f_TA_ODPe['p_value']))

F-test p-value: 0.07 



Similar to before, we take the result of the $F$-test as a lack of convincing evidence against the model. 

Thus, we now consider whether we can reduce the model by dropping the calendar effect by means of an $F$-test for
$$ H_0: \text{over-dispersed Poisson} \quad \text{vs} \quad H_A: \text{extended over-dispersed Poisson}. $$
That is, we consider a reduction from $\mu_{ij} = \alpha_i + \beta_j + \gamma_k + \delta$ to $\mu_{ij} = \alpha_i + \beta_j + \delta$.

In [19]:
model_TAe.fit_table(attach_to_self=False).loc[['AC'],:]

,deviance,df_resid,P>chi_sq,LR_vs_APC,df_vs_APC,F_vs_APC,P>F
AC,1.90301e+06,36,NaN,507496,8,1.27281,0.296797


With a p-value of $0.30$, we cannot reject this reduction. In the model without calendar effect, point forecasts match the chain-ladder technique forecasts.

We can now consider whether the model without calendar effect still survives the same tests it did before. First, we test it against a generalized log-normal model:
$$ H_0: \text{over-dispersed Poisson} \quad \text{vs} \quad H_A: \text{generalized log-normal}. $$

In [20]:
r_TA_ODP = apc.r_test(
    apc.loss_TA(), family_null='od_poisson_response', 
    predictor='AC', data_format='CL'
)
print('R-statistic: {:.2f}'.format(r_TA_ODP['R_stat']))
print('p_value: {:.4f}'.format(r_TA_ODP['p_value']))
print('Power at R: {:.2f}'.format(r_TA_ODP['power_at_R']))

R-statistic: 73.51
p_value: 0.7340
Power at R: 1.00


The model passes this test easily.

Now we can repeat the misspecification tests, testing
$$ H_{\sigma^2}: \sigma^2_\ell = \sigma^2 $$
with a Bartlett test and 
$$ H_{\mu, \sigma^2}: \alpha_{i, \ell} + \beta_{j, \ell} + \delta_\ell = \alpha_i + \beta_j + \delta $$
with an $F$-test.

In [21]:
model_TA = apc.Model()
model_TA.data_from_df(apc.loss_TA(), data_format='CL')
model_TA.fit('od_poisson_response', 'AC')

sub_models_TA = [model_TA.sub_model(per_from_to=(1,5)),
                 model_TA.sub_model(
                     coh_from_to=(1,5), age_from_to=(1,5), per_from_to=(6,10)
                 ),
                 model_TA.sub_model(age_from_to=(6,10)),
                 model_TA.sub_model(coh_from_to=(6,10))]

bartlett_TA_ODP = apc.bartlett_test(sub_models_TA)
f_TA_ODP = apc.f_test(model_TA, sub_models_TA)
print('Bartlett test p-value: {:.2f}'.format(bartlett_TA_ODP['p_value']))
print('F-test p-value: {:.2f} \n'.format(f_TA_ODP['p_value']))

Bartlett test p-value: 0.08
F-test p-value: 0.93 



These tests pass as well. Thus, we may be somewhat more comfortable in using an over-dispersed Poisson chain-ladder model to generate forecasts for this data.

## References

* Barnett, G., & Zehnwirth, B. (2000). Best estimates for reserves. *Proceedings of the Casualty Actuarial Society*, 87(167), 245–321.
* Harnau, J., & Nielsen, B. (2017). Over-dispersed age-period-cohort models. *Accepted for publication in Journal of the American Statistical Association*. https://doi.org/10.1080/01621459.2017.1366908
* Harnau, J. (2018). Log-Normal or Over-Dispersed Poisson? *Risks*, 6(3), 70.
* Harnau, J. (2018b). Misspecification Tests for Log-Normal and Over-Dispersed Poisson Chain-Ladder Models. *Risks*, 6(2), 25. 
* Kuang, D., & Nielsen, B. (2018). Generalized Log-Normal Chain-Ladder. *ArXiv E-Prints 1806.05939*. [Download](http://arxiv.org/abs/1806.05939)
* Taylor, G. C., & Ashe, F. R. (1983). Second moments of estimates of outstanding claims. *Journal of Econometrics*, 23(1), 37–61.
* Verrall, R., Nielsen, J. P., & Jessen, A. H. (2010). Prediction of RBNS and IBNR claims using claim amounts and claim counts. *ASTIN Bulletin*, 40(2), 871–887.